https://colab.research.google.com/drive/1uxoiX1h44kSc3UceUfon2oxq3SYYO-H3

In [0]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.recurrent import SimpleRNN
from keras.layers.embeddings import Embedding
from keras.layers import LSTM
import numpy as np

Using TensorFlow backend.


In [0]:
from keras.utils import to_categorical

In [0]:
#define documents
docs = ['this is','is an']
# define class labels
labels = ['an','example']


from collections import Counter
counts = Counter()
for i,review in enumerate(docs+labels):
  counts.update(review.split())
words = sorted(counts, key=counts.get, reverse=True)
vocab_size=len(words)
word_to_int = {word: i for i, word in enumerate(words, 1)}

encoded_docs = []
for doc in docs:
  encoded_docs.append([word_to_int[word] for word in doc.split()])
encoded_labels = []
for label in labels:
  encoded_labels.append([word_to_int[word] for word in label.split()])

In [0]:
word_to_int

{'an': 2, 'example': 4, 'is': 1, 'this': 3}

In [0]:
print('encoded_docs: ',encoded_docs)
print('encoded_labels: ',encoded_labels)

encoded_docs:  [[3, 1], [1, 2]]
encoded_labels:  [[2], [4]]


In [0]:
# pad documents to a max length of 2 words
max_length = 2
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='pre')
print(padded_docs)

[[3 1]
 [1 2]]


In [0]:
# processing the output dataset
one_hot_encoded_labels = to_categorical(encoded_labels, num_classes=5)
print(one_hot_encoded_labels)

[[0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]]


In [0]:
# define the model
embed_length=1
max_length=2
model = Sequential()
model.add(SimpleRNN(1,activation='tanh', return_sequences=False,recurrent_initializer='Zeros',input_shape=(max_length,embed_length),unroll=True))
model.add(Dense(5, activation='softmax'))
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())






Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 1)                 3         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 10        
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
model.fit(padded_docs.reshape(2,2,1),np.array(one_hot_encoded_labels),epochs=500)

In [0]:
model.weights

[<tf.Variable 'simple_rnn_1/kernel:0' shape=(1, 1) dtype=float32_ref>,
 <tf.Variable 'simple_rnn_1/recurrent_kernel:0' shape=(1, 1) dtype=float32_ref>,
 <tf.Variable 'simple_rnn_1/bias:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'dense_1/kernel:0' shape=(1, 5) dtype=float32_ref>,
 <tf.Variable 'dense_1/bias:0' shape=(5,) dtype=float32_ref>]

In [0]:
model.get_weights()

[array([[-0.46506366]], dtype=float32),
 array([[-0.3498325]], dtype=float32),
 array([0.3742378], dtype=float32),
 array([[-0.07940943,  1.0316525 ,  0.82078505,  0.69793195,  0.5232159 ]],
       dtype=float32),
 array([-0.40055057, -0.4714679 ,  0.43754113, -0.44841036,  0.4264429 ],
       dtype=float32)]

In [0]:
model.get_weights()[0]

array([[-0.46506366]], dtype=float32)

In [0]:
model.predict(padded_docs[0].reshape(1,2,1))

array([[0.11790387, 0.13366564, 0.31960246, 0.12894824, 0.29987988]],
      dtype=float32)

In [0]:
padded_docs[0]

array([3, 1], dtype=int32)

In [0]:
input_t0 = 3
input_t0_kernel_bias = input_t0*model.get_weights()[0] + model.get_weights()[2]

In [0]:
hidden_layer0_value = np.tanh(input_t0_kernel_bias)

In [0]:
input_t1 = 1
input_t1_kernel_bias = input_t1*model.get_weights()[0] + model.get_weights()[2]

In [0]:
input_t1_recurrent = hidden_layer0_value*model.get_weights()[1]

In [0]:
total_input_t1 = input_t1_kernel_bias + input_t1_recurrent

In [0]:
output_t1 = np.tanh(total_input_t1)

In [0]:
final_output = output_t1*model.get_weights()[3] + model.get_weights()[4]
final_output

array([[-0.41458684, -0.28911504,  0.58262146, -0.3250453 ,  0.5189255 ]],
      dtype=float32)

In [0]:
np.exp(final_output)/np.sum(np.exp(final_output))

array([[0.11790387, 0.13366562, 0.31960243, 0.12894824, 0.29987985]],
      dtype=float32)